In [3]:
%pip install torch torchvision Pillow pycocotools opencv-python azure-ai-ml tqdm scikit-learn numpy

  Using cached scikit_learn-1.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (17 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.9 MB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]
Note: you may need to restart the kernel to use updated packages.


In [6]:
# scripts/prepare_normal_crops.py

import os, glob, json
from PIL import Image

# 경로 설정
IMG_DIR  = "./data/images"
ANN_DIR  = "./data/annotations"
SAVE_ROOT = "./classification_data"

# categories 맵핑 (2개씩 짝지어 하나의 category name)
CATEGORY_MAP = {
    1: "PE드럼",   2: "PE드럼",
    3: "PE방호벽", 4: "PE방호벽",
    5: "PE안내봉", 6: "PE안내봉",
    7: "라바콘",   8: "라바콘",
    9: "시선유도봉",10:"시선유도봉",
    11:"제설함",  12:"제설함",
    13:"PE입간판",14:"PE입간판",
    15:"PE휀스",  16:"PE휀스"
}

# 저장 폴더 생성
for cat in set(CATEGORY_MAP.values()):
    os.makedirs(os.path.join(SAVE_ROOT, cat, "정상"), exist_ok=True)

# JSON 순회
for ann_path in glob.glob(os.path.join(ANN_DIR, "*.json")):
    data = json.load(open(ann_path, encoding="utf-8"))
    img_info = data["images"][0]
    img = Image.open(os.path.join(IMG_DIR, img_info["file_name"])).convert("RGB")

    for ann in data["annotations"]:
        cid = ann["category_id"]
        # 1) 홀수(id%2==1) 정상 샘플만
        if cid % 2 != 1:
            continue
        cat_name = CATEGORY_MAP[cid]
        x,y,w,h = map(int, ann["bbox"])
        crop = img.crop((x, y, x+w, y+h))

        # 파일명: {originalBase}_ann{ann_id}.jpg
        base = os.path.splitext(img_info["file_name"])[0]
        save_path = os.path.join(SAVE_ROOT, cat_name, "정상", f"{base}_ann{ann['id']}.jpg")
        crop.save(save_path)

print("정상 샘플 크롭 완료")

정상 샘플 크롭 완료


In [7]:
# scripts/prepare_anomaly_crops.py

import os, glob, json
from PIL import Image

# 경로 설정
IMG_DIR  = "./data/images"
ANN_DIR  = "./data/annotations"
SAVE_ROOT = "./classification_data"

# categories 맵핑 (2개씩 짝지어 하나의 category name)
CATEGORY_MAP = {
    1: "PE드럼",   2: "PE드럼",
    3: "PE방호벽", 4: "PE방호벽",
    5: "PE안내봉", 6: "PE안내봉",
    7: "라바콘",   8: "라바콘",
    9: "시선유도봉",10:"시선유도봉",
    11:"제설함",  12:"제설함",
    13:"PE입간판",14:"PE입간판",
    15:"PE휀스",  16:"PE휀스"
}

# 저장 폴더 생성
for cat in set(CATEGORY_MAP.values()):
    os.makedirs(os.path.join(SAVE_ROOT, cat, "파손"), exist_ok=True)

# JSON 순회
for ann_path in glob.glob(os.path.join(ANN_DIR, "*.json")):
    data = json.load(open(ann_path, encoding="utf-8"))
    img_info = data["images"][0]
    img = Image.open(os.path.join(IMG_DIR, img_info["file_name"])).convert("RGB")

    for ann in data["annotations"]:
        cid = ann["category_id"]
        # 1) 짝수(id%2==0) 파손 샘플만
        if cid % 2 == 1:
            continue
        cat_name = CATEGORY_MAP[cid]
        x,y,w,h = map(int, ann["bbox"])
        crop = img.crop((x, y, x+w, y+h))

        # 파일명: {originalBase}_ann{ann_id}.jpg
        base = os.path.splitext(img_info["file_name"])[0]
        save_path = os.path.join(SAVE_ROOT, cat_name, "파손", f"{base}_ann{ann['id']}.jpg")
        crop.save(save_path)

print("파손 샘플 크롭 완료")

파손 샘플 크롭 완료


In [1]:
import os
import cv2
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as T
from sklearn.model_selection import train_test_split

# 1. 데이터 수집 함수
def load_images_and_labels(root_dir, img_size=(224,224)):
    images, labels = [], []
    # classification_data/{category}/{정상,파손}/
    for category in os.listdir(root_dir):
        cat_path = os.path.join(root_dir, category)
        for state, label in [("정상", 0), ("파손", 1)]:
            folder = os.path.join(cat_path, state)
            if not os.path.isdir(folder):
                continue
            for fname in os.listdir(folder):
                path = os.path.join(folder, fname)
                img = cv2.imread(path)
                if img is None:
                    continue
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, img_size)
                images.append(img.astype(np.float32)/255.0)
                labels.append(label)
    return np.array(images), np.array(labels)

# 2. Dataset 클래스
class ImgFolderDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        img = self.images[idx]
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx]

In [2]:
# 3. Device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 4. 데이터 불러오기 및 분할
ROOT = os.path.join("./classification_data")
IMG_SIZE = (224,224)

X, y = load_images_and_labels(ROOT, IMG_SIZE)
# train/test 8:2 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print(f"Train: {len(X_train)} images, Val: {len(X_val)} images")

# 5. Transform 정의
transform = T.Compose([
    T.ToTensor(),  # HWC -> CHW, [0,1]
    T.Normalize(mean=[0.485,0.456,0.406],
                std =[0.229,0.224,0.225])
])

# 6. DataLoader 생성
batch_size = 4
train_ds = ImgFolderDataset(X_train, y_train, transform)
val_ds   = ImgFolderDataset(X_val,   y_val,   transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                          num_workers=8, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False,
                          num_workers=8, pin_memory=True)

# 7. 모델 준비
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)  # 이진 분류
model = model.to(device)

# 8. 손실 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 9. 학습/검증 루프
num_epochs = 15
for epoch in range(1, num_epochs+1):
    model.train()
    running_loss = 0.0
    for imgs, lbls in train_loader:
        imgs, lbls = imgs.to(device), lbls.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, lbls)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    epoch_loss = running_loss / len(train_ds)

    # 검증
    model.eval()
    correct = 0
    with torch.no_grad():
        for imgs, lbls in val_loader:
            imgs, lbls = imgs.to(device), lbls.to(device)
            preds = model(imgs).argmax(dim=1)
            correct += (preds == lbls).sum().item()
    val_acc = correct / len(val_ds)

    print(f"Epoch {epoch}/{num_epochs} • "
          f"Train Loss: {epoch_loss:.4f} • Val Acc: {val_acc:.4f}")

# 10. 모델 저장
os.makedirs("outputs", exist_ok=True)
torch.save(model.state_dict(), "./outputs/resnet_binary.pt")
print("Model saved → outputs/resnet_binary.pt")

Using device: cuda


Train: 18603 images, Val: 4651 images
Epoch 1/15 • Train Loss: 0.0903 • Val Acc: 0.9804
Epoch 2/15 • Train Loss: 0.0257 • Val Acc: 0.9912
Epoch 3/15 • Train Loss: 0.0181 • Val Acc: 0.9976
Epoch 4/15 • Train Loss: 0.0141 • Val Acc: 0.9908
Epoch 5/15 • Train Loss: 0.0131 • Val Acc: 0.9983
Epoch 6/15 • Train Loss: 0.0113 • Val Acc: 0.9918
Epoch 7/15 • Train Loss: 0.0079 • Val Acc: 0.9959
Epoch 8/15 • Train Loss: 0.0078 • Val Acc: 0.9970
Epoch 9/15 • Train Loss: 0.0057 • Val Acc: 0.9912
Epoch 10/15 • Train Loss: 0.0076 • Val Acc: 0.9856
Epoch 11/15 • Train Loss: 0.0048 • Val Acc: 0.9929
Epoch 12/15 • Train Loss: 0.0046 • Val Acc: 0.9961
Epoch 13/15 • Train Loss: 0.0056 • Val Acc: 0.9897
Epoch 14/15 • Train Loss: 0.0037 • Val Acc: 0.9976
Epoch 15/15 • Train Loss: 0.0062 • Val Acc: 0.9970
Model saved → outputs/resnet_binary.pt


In [5]:
# 11. 단일 이미지 예측 함수
def predict(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE).astype(np.float32)/255.0
    tensor = transform(img).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        out = model(tensor)
        prob = nn.Softmax(dim=1)(out)[0]
        cls = prob.argmax().item()
    return cls, prob.cpu().numpy()

# 예시
cls, prob = predict("./test2.png")
print("Pred:", cls, "Prob:", prob)

Pred: 1 Prob: [0.14724366 0.8527564 ]
